# Beneficiários

In [1]:
import os
import requests

import pandas as pd
import numpy as np

from io import BytesIO
from collections import defaultdict

### Códigos ANS (RI Hapvida)

In [2]:
# donwload dos códigos ANS de Hapvida e GNDI

RI_URL = 'https://api.mziq.com/mzfilemanager/v2/d/6bbd1770-f9f4-44e8-a1b1-d26b7585eec1/6d20a378-c816-6a0a-ad88-d557761215bd?origin=1'

def ans_codes(url, empresa):
    response = requests.get(url)
    content = response.content

    if empresa == 'HAPVIDA':
        df = pd.read_excel(BytesIO(content), sheet_name='Códigos ANS_ANS Code', skiprows=1, nrows=28, usecols=[0,1,2,3,4])
        df.columns = ['CD_ANS', 'EMPRESA', 'GRUPO', 'ASSINATURA', 'FECHAMENTO']

    elif empresa == 'GNDI':
        df = pd.read_excel(BytesIO(content), sheet_name='Códigos ANS_ANS Code', skiprows=1, nrows=33, usecols=[6, 7, 8, 9])
        df.columns = ['CD_ANS', 'EMPRESA', 'ASSINATURA', 'FECHAMENTO']

    else:
        raise ValueError("Invalid empresa value. Must be 'HAPVIDA' or 'GNDI'.")

    return df

In [3]:
# download hapvida ANS codes

hapvida = ans_codes(RI_URL, 'HAPVIDA')
hapvida_codes = hapvida['CD_ANS'].astype('int32')

hapvida_codes.shape


(28,)

In [4]:
intermedica = ans_codes(RI_URL, 'GNDI')

intermedica_codes = intermedica['CD_ANS'].dropna()
intermedica_codes = intermedica_codes.iloc[:-1].tolist() + intermedica_codes.iloc[-1].split('/')
intermedica_codes = pd.Series(intermedica_codes).astype('int32')

intermedica_codes.shape


(20,)

In [5]:
# combine all closings codes

combined_codes = pd.concat([hapvida_codes, intermedica_codes], ignore_index=True)

combined_codes.shape

(48,)

### Market Share

In [6]:
# download SIB data
types = defaultdict(lambda: 'int32')

data = pd.concat([pd.read_csv(f'data/{file}', dtype=types) for file in os.listdir('data') if os.path.getsize(f'data/{file}') > 0], ignore_index=True)
data = data.astype(np.int32)

data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48053419 entries, 0 to 48053418
Data columns (total 4 columns):
 #   Column                 Dtype
---  ------                 -----
 0   #ID_CMPT_MOVEL         int32
 1   CD_OPERADORA           int32
 2   COBERTURA_ASSIST_PLAN  int32
 3   QT_BENEFICIARIO_ATIVO  int32
dtypes: int32(4)
memory usage: 733.2 MB


In [7]:
data.loc[data['COBERTURA_ASSIST_PLAN'] == 1, :].groupby('#ID_CMPT_MOVEL')['QT_BENEFICIARIO_ATIVO'].sum()

#ID_CMPT_MOVEL
202206    49523045
202209    49899062
202212    50108199
Name: QT_BENEFICIARIO_ATIVO, dtype: int32

In [8]:
filtered_data = data.loc[(data['COBERTURA_ASSIST_PLAN'] == 1) & (data['#ID_CMPT_MOVEL'] == 202212), :].copy()

filtered_data['CAT_OPERADORA'] = np.where(filtered_data['CD_OPERADORA'].isin(combined_codes), 1, 0)

grouped_data = filtered_data.groupby('CAT_OPERADORA')['QT_BENEFICIARIO_ATIVO'].sum()

percentage_data = grouped_data / grouped_data.sum() * 100

print(f'Porcentagem de beneficiários Hapvida: {percentage_data[1]:.2f}%')


Porcentagem de beneficiários Hapvida: 10.91%


In [9]:
filtered_data = data.loc[(data['COBERTURA_ASSIST_PLAN'] == 1) & (data['#ID_CMPT_MOVEL'] == 202212), :].copy()

filtered_data['CAT_OPERADORA'] = np.where(filtered_data['CD_OPERADORA'].isin(combined_codes), 1, 0)

grouped_data = filtered_data.groupby('CAT_OPERADORA').size()

percentage_data = grouped_data / grouped_data.sum() * 100

print(f'Porcentagem de planos Hapvida: {percentage_data[1]:.2f}%')

Porcentagem de planos Hapvida: 8.56%
